In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 19 19:58:10 2017

@author: Lesley
"""
from math import log
#------------------------数据集---------------------------------------------
#四个因变量（属性） -> feature : "Outlook","Temperature","Humidity","Windy"
#标签 -> 是否出门 -> 'no','yes'
#---------------------------------------------------------------------------
def createDataSet():
    dataSet=[["sunny","hot","high",'FALSE','no'],
            ["sunny","hot","high",'TRUE','no'],
            ["overcast","hot","high",'FALSE','yes'],
            ["rainy","mild","high",'FALSE','yes'],
            ["rainy","cool","normal",'FALSE','yes'],
            ["rainy","cool","normal",'TRUE','no'],
            ["overcast","cool","normal",'TRUE','yes'],
            ["sunny","mild","high",'false','no'],
            ["sunny","cool","normal",'false','yes'],
            ["rainy","mild","normal",'false','yes'],
            ["sunny","mild","normal",'true','yes'],
            ["overcast","mild","high",'TRUE','yes'],
            ["overcast","hot","normal",'false','yes'],
            ["rainy","mild","high",'TRUE','no']
            ]
    feature = ["Outlook","Temperature","Humidity","Windy"]
    return dataSet, feature

#----------------计算数据集整体及各个feature的不同值的熵-------------------
def Entropy(dataSet):
    sample_num = len(dataSet)    
    label_category = {}
    for sample in dataSet:
        if sample[-1] not in label_category:
            label_category[sample[-1]] = 1
        else:
            label_category[sample[-1]] += 1  #各类别的数量
    #print (label_category)
    entropy = 0
    for i in label_category:
        temp = label_category[i]/sample_num
        entropy -= temp * log(temp,2)
    return entropy

#----------------根据不同feature的不同值筛选对应的数据集-------------------
def Extractdata(dataSet, axis, value):       #筛选出的是第axis个feature中值为value的sample集合 且此集合中不含第axis个featur对应的数据
    extDataSet = []
    for sample in dataSet:
        if sample[axis] == value:
            extSample = sample[:axis]        #复制第axis个featur之前的数据
            extSample.extend(sample[axis+1:])#复制第axis个featur之后的数据
            extDataSet.append(extSample)     #删除第axis个featur对应的数据
    return extDataSet

#------------对于每一节点得到最大的信息增益及对应的feature-----------------
def BestFeature(dataSet):
    feature_num = len(dataSet[0]) - 1     #数据集的最后一项是标签
    entropy = Entropy(dataSet)
    bestEntropy = 0
    bestInfoGain = 0
    bestFeature = -1
    for i in range(feature_num):
        featuresample = [example[i] for example in dataSet]
        uniqueVals = set(featuresample)   #去除list中的重复元素
        newEntropy = 0
        for value in uniqueVals:
            extDataSet = Extractdata(dataSet, i, value)
            prob = len(extDataSet) / float(len(dataSet))  #当前feature中各个值所占比重
            newEntropy += prob * Entropy(extDataSet)      #Entropy(extDataSet) 得到的是第i个feature中的第j个值的熵

        gain = entropy - newEntropy       #信息增益越大越好 -> 熵越小，数据越纯净
        
        if gain > bestInfoGain:
            bestInfoGain = gain
            bestFeature = i
            bestEntropy = entropy - gain
    return bestInfoGain, bestFeature, bestEntropy

#--------------------------------多数表决--------------------------------
def majorityCnt(classList):  
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    return max(classCount)     #classlist中出现最多的元素      

#-------------------------------建树-------------------------------------
def CreateTree(dataSet,feature):
    classList = [example[-1] for example in dataSet]  #标签项
    best_infogain, best_feature, best_entropy= BestFeature(dataSet)  #最大信息增益增益值与其所对应的feature的顺序
    
    #------------------------停止划分的条件------------------------------
    #数据集中的最后一项：标签为同一个值（类别相同）   best_entropy-> 最大熵为0
    if classList.count(classList[0]) ==len(classList): 
        #print(best_entropy)
        return classList[0]
    
    #最大信息熵小于等于0
    if best_infogain < 0 or best_infogain == 0:
        return majorityCnt(classList)
    
    #所有特征已经用完 -> 剩余标签
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)          #剩余的list中所有feature已被用完，但标签中仍没有一致，这是采用多数表决
    #-------------------------------------------------------------------
        
    best_feature_name = feature[best_feature]  #最大信息增益对应的feature的名称
    myTree = {best_feature_name:{}}            #根节点 -> 某一个feature
    
    del(feature[best_feature])                 #删去已经处理过的feature属性
    featuresample = [example[best_feature] for example in dataSet]
    uniqueVals = set(featuresample)            #对于每一个feature的各个值
    
    for value in uniqueVals:
        subfeature = feature[:]#复制，使进一步的调用函数不会对原feature产生影响。 python中函数传入的list参数若修改会影响其原始值
        myTree[best_feature_name][value] = CreateTree( Extractdata(dataSet, best_feature, value),subfeature)
    return myTree
   
def main():
    data, feature = createDataSet()
    myTree = CreateTree(data,feature)
    print (myTree)
    
if __name__=='__main__':
    main()

{'Outlook': {'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}, 'rainy': {'Windy': {'false': 'yes', 'TRUE': 'no', 'FALSE': 'yes'}}, 'overcast': 'yes'}}
